In [1]:

import pandas as pd
import folium
from geopy.distance import geodesic

# Load the data
data = pd.read_excel('supply_chain_network.xlsx')

# Filter suppliers and facilities
suppliers = data[data['Type'] == 'Supplier']
facilities = data[data['Type'] == 'Facility']

# Initialize lists to store distances and emissions
distances = []
emissions = []

# Calculate distances and emissions
s = 1
j = 1
for _, supplier in suppliers.iterrows():
    for _, facility in facilities.iterrows():
        distance = geodesic((supplier['Latitude'], supplier['Longitude']), (facility['Latitude'], facility['Longitude'])).kilometers
        emission = distance * 0.0079  # 7.9 g/ton-km converted to tons
        cost = distance * 0.1 # Assuming a cost of $0.1 per km per ton
        distances.append({
            's': s,
            'j': j,
            'Supplier': supplier['ID'],
            'Facility': facility['ID'],
            'Distance (km)': distance,
            'CO2 Emissions (tons)': emission,
            'cost' : cost
        })
        j += 1
    s += 1
    j = 1

# Convert to DataFrame
distances_df = pd.DataFrame(distances)

# Save distances and emissions to Excel
distances_df.to_excel('supplier_facility_distances_emissions.xlsx', index=False)

# Create a map centered around the midpoint of the locations
m = folium.Map(location=[30, 100], zoom_start=3)

# Add facility markers
for _, facility in facilities.iterrows():
    folium.Marker(
        location=[facility['Latitude'], facility['Longitude']],
        popup=f'Facility {facility["ID"]} ({facility["Country"]})',
        icon=folium.Icon(color='blue', icon='industry', prefix='fa')
    ).add_to(m)

# Add supplier markers
for _, supplier in suppliers.iterrows():
    folium.Marker(
        location=[supplier['Latitude'], supplier['Longitude']],
        popup=f'Supplier {supplier["ID"]} ({supplier["Country"]})',
        icon=folium.Icon(color='green', icon='truck', prefix='fa')
    ).add_to(m)

# Add a legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 150px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; opacity: 0.85;">
     &nbsp; <i class="fa fa-industry fa-2x" style="color:blue"></i>&nbsp; Facilities<br>
     &nbsp; <i class="fa fa-truck fa-2x" style="color:green"></i>&nbsp; Suppliers<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
m.save("supply_chain_network_map_with_legend.html")
